<a href="https://colab.research.google.com/github/AnviTendulkar/Airline_Fare_Prediction/blob/master/final3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import re
from sklearn.model_selection import KFold ,cross_val_score ,GridSearchCV
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PowerTransformer ,OneHotEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge ,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor ,ExtraTreesRegressor ,AdaBoostRegressor ,GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

In [2]:
pd.set_option("display.max_columns" ,None)
pd.set_option("display.max_rows" ,None)

In [3]:
train_data = pd.read_excel(r'/content/drive/MyDrive/Project/Airline Fare  Prediction/Data_Train.xlsx')

In [4]:
train_data.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


In [5]:
train_data.dropna(inplace=True)

In [6]:
X = train_data.loc[:,train_data.columns !='Price']
y =train_data['Price']

In [7]:
# print(X.shape ,y.shape)
# print(x_train.shape ,x_test.shape,y_train.shape,y_test.shape ,df.shape,y.shape)

In [8]:
X[X.Airline =='Trujet']

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
2878,Trujet,6/03/2019,Mumbai,Hyderabad,BOM → NDC → HYD,13:05,16:20,3h 15m,1 stop,No info


In [9]:
# pip install scikit-multilearn

In [10]:
# pip install -U scikit-multilearn


In [11]:
# import skmultilearn
# from skmultilearn.model_selection import iterative_train_test_split
# x_train, y_train, x_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)
# ValueError: not enough values to unpack (expected 2, got 1)

x_train ,x_test ,y_train ,y_test = train_test_split(X,y ,test_size=0.20 )

In [12]:
def Arrival_Date_calculation(row):
    pattern = re.compile(r'(?:(\d+)h)?\s?(?:(\d+)m)?')
    match = pattern.match(row['Duration'])
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0

    # Concatenate 'Date_of_Journey' and 'Dep_Time'
    date_of_journey = row['Date_of_Journey'] + ' ' + row['Dep_Time']

    # Convert the concatenated string to datetime
    journey_date_time = datetime.strptime(date_of_journey, "%d/%m/%Y %H:%M")

    # Calculate arrival date by adding 'Duration_hour' and 'Duration_minute' to 'Date_of_Journey'
    arrival_date = journey_date_time + timedelta(hours=hours, minutes=minutes)
    return arrival_date

def split_dep_time_in_hours_min(str1):
    lst = str1.split(':')
    return pd.Series({'Dep_Hour': lst[0], 'Dep_Minute': lst[1]})

In [13]:
###################
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class DateColumnExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        temp_df = pd.DataFrame()
        X['Arrival_Date'] = X.apply(Arrival_Date_calculation, axis=1)
        temp_df['Arrival_Year'] = X['Arrival_Date'].dt.year
        temp_df['Arrival_Month'] = X['Arrival_Date'].dt.month
        temp_df['Arrival_Day'] = X['Arrival_Date'].dt.day
        temp_df['Arrival_Hour'] = X['Arrival_Date'].dt.hour
        temp_df['Arrival_Minute'] = X['Arrival_Date'].dt.minute

        temp = X['Date_of_Journey'].copy()
        temp = pd.to_datetime(temp ,format='%d/%m/%Y')
        # Extract year, month, and day into separate columns
        temp_df['Dep_Year'] = temp.dt.year
        temp_df['Dep_Month'] = temp.dt.month
        temp_df['Dep_Day'] = temp.dt.day

        temp_df[['Dep_Hour', 'Dep_Minute']] = X['Dep_Time'].apply(split_dep_time_in_hours_min)
        return temp_df

    def get_params(self, deep=True):
        return {}


In [14]:
class TotalStopConverter(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        temp_df = pd.DataFrame()
        df_stop= X.replace({'1 stops': 1, '1 stop': 1, 'non-stop': 0, '2 stops': 2, '3 stops': 3, '4 stops': 4})

        return df_stop

    def get_params(self, deep=True):
        return {}

In [15]:
#################################################

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
import pandas as pd

# Assume the DateColumnExtractor class is defined as provided in your previous message

# Your DataFrame
df = x_train[['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Dep_Time', 'Duration', 'Total_Stops']]

# Define the DateColumnExtractor
date_extractor = DateColumnExtractor()

# Define the ColumnTransformer
preprocessor = ColumnTransformer([
    ('one-hot', OneHotEncoder(drop='first', sparse_output=False, dtype='int16' ,handle_unknown = 'ignore'), [0, 2, 3]),
    # ('total_stop', FunctionTransformer(lambda x: x.replace({'1 stops': 1, '1 stop': 1, 'non-stop': 0, '2 stops': 2, '3 stops': 3, '4 stops': 4}),
    #                                   validate=False), [df.columns.get_loc('Total_Stops')]),
    ('TotalStopConverter', TotalStopConverter, ['Total_Stops']),
    ('date_extractor', date_extractor, ['Date_of_Journey', 'Dep_Time','Duration']),

], remainder='passthrough')

# Define the pipeline
# pipe_clf = Pipeline([
#     ('preprocessor', preprocessor),
#     ('scaling' ,StandardScaler()),
#     ('regression_model', LinearRegression())
# ])


clf = Pipeline([
    ('preprocessor', preprocessor),
    ('scaling' ,StandardScaler()),
    # ('regression_model', LinearRegression())
])
pipe_clf = Pipeline([
    ("clf" ,clf),
    ('regression_model', LinearRegression())
])

# Fit and transform the data
# transformed_data = pipe_clf.fit_transform(df) //if linear regression is used then fit method does not work
transformed_data = pipe_clf.fit(df,y_train)
# print(pd.DataFrame(transformed_data).head(3))



TypeError: ignored

In [ ]:
predictions = pipe_clf.predict(x_test)
from sklearn.metrics import mean_absolute_error
print("Model Accuracy: %.3f" % pipe_clf.score(x_test, y_test))
mae = mean_absolute_error(y_test, predictions)
print("The mean squared error (MSE) on test set: {:.4f}".format(mae))

In [ ]:
print(x_train['Airline'].unique())
print(x_test['Airline'].unique())

In [ ]:
# Get the names of the transformed columns
one_hot_columns = preprocessor.named_transformers_['one-hot'].get_feature_names_out(['Airline', 'Source', 'Destination'])
total_stop_columns = ['Total_Stops']  # You might need to update this based on your actual transformation

# Combine all transformed column names
transformed_columns = list(one_hot_columns) + total_stop_columns + ['Arrival_Year', 'Arrival_Month', 'Arrival_Day', 'Arrival_Hour', 'Arrival_Minute', 'Dep_Year', 'Dep_Month', 'Dep_Day', 'Dep_Hour', 'Dep_Minute']

# Concatenate the transformed data with the remaining columns and set column names
transformed_df = pd.DataFrame(clf.transform(df) ,columns = transformed_columns)

# Print the transformed DataFrame with column names
print(transformed_df.head(3))


In [ ]:
### predict one item
data = [['Jet Airways' ,'15/05/2019' ,'Banglore' ,'Delhi','15:15' ,'2h 55m' ,'non-stop']]
columns = ['Airline'	,'Date_of_Journey',	'Source',	'Destination',	'Dep_Time',	'Duration',	'Total_Stops']
# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)
pipe_clf.predict(one_df)

In [ ]:
df.head(1)

In [ ]:
y_train.head(1)

In [ ]:
### Test the different models

In [ ]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [ ]:
# preprocessor = ColumnTransformer([
#     ('one-hot', OneHotEncoder(drop='first', sparse_output=False, dtype='int16'), [0, 2, 3]),
#     ('total_stop', FunctionTransformer(lambda x: x.replace({'1 stops': 1, '1 stop': 1, 'non-stop': 0, '2 stops': 2, '3 stops': 3, '4 stops': 4}),
#                                       validate=False), [df.columns.get_loc('Total_Stops')]),
#     ('date_extractor', date_extractor, ['Date_of_Journey', 'Dep_Time','Duration']),

# ], remainder='passthrough')

def scorer(model_name, model):

    output = []

    output.append(model_name)

    # pipeline = Pipeline([
    #     ('preprocessor', preprocessor),
    #     ('regressor', model)
    # ])

    clf = Pipeline([
    ('preprocessor', preprocessor),
    ('scaling' ,StandardScaler()),
    # ('regression_model', LinearRegression())
    ])
    pipe_clf = Pipeline([
    ("clf" ,clf),
    ('regressor', model)
    ])
    df = X[['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Dep_Time', 'Duration', 'Total_Stops']]
    print(df.shape ,y.shape)
    X_train, X_test, y_train, y_test = train_test_split(df,y,test_size=0.2,random_state=42)
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipe_clf,X_train, y_train , cv=kfold, scoring='r2')

    output.append(scores.mean())



    pipe_clf.fit(X_train,y_train)

    y_pred = pipe_clf.predict(X_test)

    output.append(mean_absolute_error(y_test,y_pred))

    return output


In [ ]:
# model_output = []
# for model_name,model in model_dict.items():
#     model_output.append(scorer(model_name, model))
# model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])
# model_df.sort_values(['mae'])
# model_df

### parameter HyperParameterTuning

In [ ]:
param_grid = {'xgb__n_estimators': [100, 200, 300],
    'xgb__learning_rate': [0.01, 0.1, 0.2],
    'xgb__max_depth': [3, 4, 5]
              }

In [ ]:
  pipeline = Pipeline([
    ("clf" ,clf),
    ('xgb', XGBRegressor())
    ])

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', verbose=4)

In [ ]:
grid_search.fit(df,y_train)
best_params = grid_search.best_params_
print(best_params)
best_estimator = grid_search.best_estimator_
print(best_estimator)

In [ ]:
grid_search.best_params_

In [ ]:
param_grid1 = {'n_estimators': 150,
    'learning_rate': 0.2,
    'max_depth': 5}
final_pipeline = Pipeline([
    ('clf', clf),
    ('xgb', XGBRegressor(**param_grid1))
])


In [ ]:
X = X[['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Dep_Time', 'Duration', 'Total_Stops']]
x_train,x_test,y_train,y_test = train_test_split(X,y ,test_size=0.2 ,random_state=42)
# search = GradientBoostingRegressor(**param_grid)
final_pipeline.fit(x_train ,y_train)
from sklearn.metrics import mean_absolute_error
print("Model Accuracy: %.3f" % final_pipeline.score(x_test, y_test))
mae = mean_absolute_error(y_test, final_pipeline.predict(x_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mae))

## predict output

In [ ]:
data = [['Jet Airways' ,'15/05/2019' ,'Banglore' ,'Delhi','15:15' ,'2h 55m' ,'non-stop']]
columns = ['Airline'	,'Date_of_Journey',	'Source',	'Destination',	'Dep_Time',	'Duration',	'Total_Stops']
# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)
final_pipeline.predict(one_df)

In [ ]:
x_train[(x_train['Airline'] =='Jet Airways') & (x_train['Date_of_Journey'] =='15/05/2019') & (x_train['Dep_Time'] =='15:15')]

In [ ]:
y_train.iloc[8417]

In [ ]:
X.iloc[[1420,8417]]

In [ ]:
import pickle
with open('final_pipeline' ,'wb')as file:
  pickle.dump(final_pipeline ,file ,protocol=2)